In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error


#Read and describe data get columns
data = pd.read_csv('melb_data.csv')
#data.describe()
data.head()


,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [43]:
#Look at columns, assign prediction target price 
data = data.dropna(axis=0)
print(data.columns)

#Assign prediction target
y = data.Price

#Assign features
data_features = ['Rooms', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea',
                  'YearBuilt', 'Lattitude', 'Longtitude']
X = data[data_features]


Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')


In [44]:
#Split training data
trainX, valX, trainY, valY = train_test_split(X, y, random_state=1)

#initial model
data_model = RandomForestRegressor(random_state=1)
data_model.fit(trainX, trainY)
prediction_value = data_model.predict(valX)
print("Prediciton values", prediction_value)

Prediciton values [ 609591.         1624620.          926225.         ... 1746095.
  793884.16666667 1608075.        ]


In [45]:
#Calculate mean absolute error of initial values 
initial_mae = mean_absolute_error(valY, prediction_value)
print(initial_mae)


174198.78252328688


In [ ]:
#Cell determines best leaf size for model

#Make function to calculate mae
def get_mae(max_leaf_nodes, trainX, valX, trainY, valY):
    model = RandomForestRegressor(random_state=1)
    model.fit(trainX, trainY)
    prediction_val = model.predict(valX)
    mae = mean_absolute_error(valY, prediction_val)
    return mae
    print(mae)

#Get max leaf nodes, store mae in scores dict, get min mae leaf
leaf_nodes = [5, 50, 100, 250, 500, 1000]
scores = {}
for max_leaf_nodes in leaf_nodes:
    my_mae = get_mae(max_leaf_nodes, trainX, valX, trainY, valY)
    scores[max_leaf_nodes] = my_mae
    best_leaf = min(scores, key=scores.get)

print(best_leaf)

5


In [56]:
#Use all data to train with new best tree size
final_model = RandomForestRegressor(max_leaf_nodes = best_leaf, random_state=1)
final_model.fit(X, y)
predictions_final = final_model.predict(X)

results = pd.DataFrame({
    'Predictions': predictions_final,
    'Actual': y
})

print(results.head(10))

     Predictions     Actual
1   1.205913e+06  1035000.0
2   1.233848e+06  1465000.0
4   7.911792e+05  1600000.0
6   1.858668e+06  1876000.0
7   1.220993e+06  1636000.0
9   1.205913e+06  1097000.0
11  1.223292e+06  1350000.0
12  7.068979e+05   750000.0
15  1.213222e+06  1310000.0
16  1.220993e+06  1200000.0
